In [2]:
import numpy as np

from utils import *

import warnings
warnings.filterwarnings('ignore')

### Find Model And Input Image Homography Matrice

In [3]:
# We found corners in section Part1; so, I don't have to find again.
model_joined_topbot = [[5, 3], [695, 3]] + [[5, 379], [695, 379]]
model_img = cv.imread('images/0.jpg')

In [4]:
image_file = './images/1_man.jpg'
default_width = 700
frame_raw = cv.imread(image_file)

if frame_raw.shape[1] > default_width:
    frame_raw = image_resizer(image_file, default_width)
    cv.imwrite("./output-part2/resized-image.jpg", frame_raw)

cv.imwrite("./output-part2/input-gray.jpg", cv.cvtColor(frame_raw, cv.COLOR_BGR2GRAY))

src = frame_raw
dst = cv.Canny(src, 50, 200, None, 3)
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

cv.imwrite("./output-part2/input-canny.jpg", dst)


frame_lines = dst
cartesian_lines = []
lines = cv.HoughLines(dst, 1, np.pi / 180, 110, None, 0, 0)

#Draw hough Lines

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
        
cv.imwrite("./output-part2/input-image-hough-lines.jpg", cdst)

if lines is not None:
    for i in range(0, len(lines)):
        pt1, pt2 = add_line(lines, i, frame_lines)
        cartesian_lines.append([[pt1[0], pt1[1]], [pt2[0], pt2[1]]])

junction_points = junctions(cartesian_lines)
junction_points_normalized = []

for junction_point in junction_points:
    if (junction_point[0] > 0 and junction_point[1] > 0):
        if (junction_point[0] < src.shape[1] and junction_point[1] > src.shape[0]):
            junction_points_normalized.append(junction_point)

cluster_centers = kmeans_centers(junction_points, 25)
mark_all(junction_points, frame_lines, rbg=(0, 0, 255))

printed = ''
for cluster_center in cluster_centers:
    printed += str(cluster_center) + ', '

print(printed)

warped_img = []
warped_org_image = []
n_noninf = get_noninf(cluster_centers)
if (len(n_noninf) >= 4):
    top, bot = get_frame_corners(n_noninf)
    frame_joined_topbot = top + bot
    if (len(frame_joined_topbot) == 4):
        # Calculate the mask for the warping
        mask, status = cv.findHomography(np.array(frame_joined_topbot), np.array(model_joined_topbot))
        print("Mask:\n", mask)
        # Get frame resolution for cv.warpPerspective()
        height, width, channels = model_img.shape
        warped_img = cv.warpPerspective(frame_raw, mask, (width, height))
        
        inv_h = np.linalg.inv(mask)
        warped_org_image = cv.warpPerspective(warped_img, inv_h, (frame_raw.shape[1], frame_raw.shape[0]), flags=cv.INTER_AREA, borderMode=cv.BORDER_CONSTANT, borderValue=(255, 255, 255))
        
    mark_all(n_noninf, frame_lines)
    print("Clusters: ", len(n_noninf))  # Number of cluster centers on screen

# cv.imshow("Model", warped_img)
cv.imwrite("./output-part2/warped_image.jpg", warped_img)
cv.imwrite("./output-part2/warped_org_image.jpg", warped_org_image)
cv.waitKey(0)
cv.destroyAllWindows()


[ 588.         -477.17307692], [-53548. -12114.], [-25988.  -4045.], [43417.  6145.], [-3309.6  -510. ], [265.     368.8125], [-1984.6  -203.5], [-5666.25  -938.5 ], [-2667.42857143  -337.5       ], [-23392.  -4055.], [555.94444444 385.83333333], [565.52173913 133.69565217], [-34.92307692 426.92307692], [ 67.51282051 272.84615385], [199.52631579  87.31578947], [-26316.  -5820.], [ 889.5 -932.5], [-1149.  -145.], [-3693.4  -563.6], [-2364.08333333  -273.25      ], [-4147.5  -510.5], [-2946.16666667  -408.83333333], [ 1279. -1468.], [367.61904762 112.42857143], [548.5 600. ], 
D/E:  2.27086062791827
Points on top: [array([199.52631579,  87.31578947]), array([565.52173913, 133.69565217])]
Points on bot: [array([ 67.51282051, 272.84615385]), array([555.94444444, 385.83333333])]
Mask:
 [[ 2.61280309e+00  1.87087799e+00 -6.78266530e+02]
 [-4.03975065e-01  3.20257258e+00 -1.95183303e+02]
 [ 2.96542741e-04  2.56037551e-03  1.00000000e+00]]
Clusters:  6


In [60]:
image = cv.imread('./output-part2/warped_image.jpg')

colored_image = np.copy(image)

for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        
        pixel_value = image[i, j]
        if ((np.mean(pixel_value) > 130 and np.mean(pixel_value) < 200)):
            colored_image[i, j] = [0, 0, 0] # Set black

cv.imwrite("./output-part2/warped-with-colored-image.jpg", colored_image)

# i > 384 - yükseklik
# i > 702 - genişlik
circle_widths = []

for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        pixel_value = image[i, j]
        if (np.mean(pixel_value) < 60):
            circle_widths.append(np.mean(pixel_value))
            
circle_width = np.mean(circle_widths)
circle_i = 0
circle_j = 0

for j in range(image.shape[1]):
    player_height = 0
    for i in range(image.shape[0]):
        pixel_value = image[i, j]
        if np.mean(pixel_value) > 46 and np.mean(pixel_value) < 48:
            player_height = player_height + 1
            if player_height > 50:
                circle_i = i
                circle_j = j
        
center_coordinates = (circle_j, circle_i)  
radius = round(circle_width)
color = (0, 0, 255) 
thickness = -1
colored_image = cv.circle(colored_image, center_coordinates, radius, color, thickness) 
cv.imwrite("./output-part2/colored-image.jpg", colored_image)

final_image = np.copy(image)
for i in range(colored_image.shape[0]):
    for j in range(colored_image.shape[1]):
        pixel_value = colored_image[i, j]
        if np.mean(pixel_value) == 0 or pixel_value[2] == 255:
            final_image[i,j] = colored_image[i,j]
            
cv.imwrite("./output-part2/final-image.jpg", final_image)


            

True

### I can use this github page

https://github.com/mokumus/OpenCV-Homography/tree/master